In [ ]:
!pip install pandas numpy SpeechRecognition transformers gtts pydub
!apt-get install -y ffmpeg

In [ ]:
import os
import speech_recognition as sr
from transformers import pipeline
from gtts import gTTS
from google.colab import drive
from google.colab import files
from pydub import AudioSegment
from IPython.display import Audio, display

sentiment_pipeline = pipeline("sentiment-analysis")

def convert_to_wav(uploaded_file_path, output_file_path):
    audio = AudioSegment.from_file(uploaded_file_path)
    audio.export(output_file_path, format="wav")

def speech_to_text():
    recognizer = sr.Recognizer()
    print("Please upload your audio file.")
    uploaded = files.upload()
    uploaded_file_path = list(uploaded.keys())[0]
    wav_file_path = "output.wav"

    try:
        convert_to_wav(uploaded_file_path, wav_file_path)
        with sr.AudioFile(wav_file_path) as source:
            audio = recognizer.record(source)
            try:
                text = recognizer.recognize_google(audio)
                print(f"You said: {text}")
                return text
            except sr.UnknownValueError:
                print("Google Speech Recognition could not understand audio")
            except sr.RequestError as e:
                print(f"Could not request results from Google Speech Recognition service; {e}")
    except Exception as e:
        print(f"Error processing audio file: {e}")
    return None

def text_to_speech(text):
    tts = gTTS(text)
    tts.save("response.mp3")
    display(Audio("response.mp3", autoplay=True))

def analyze_sentiment(text):
    results = sentiment_pipeline(text)
    sentiment = results[0]['label']
    return sentiment

def sentiment_analysis():
    print("System is ready to analyze sentiment! (Say 'exit' to stop)")
    while True:
        text = speech_to_text()
        if text is not None:
            if text.lower() == 'exit':
                print("Goodbye!")
                text_to_speech("Goodbye!")
                break
            sentiment = analyze_sentiment(text)
            response = f"Your sentence is {sentiment.lower()}."
            print(f"System: {response}")
            text_to_speech(response)
        else:
            text_to_speech("Sorry, I didn't catch that. Could you please repeat?")

sentiment_analysis()